# Data Analytics Project Work
### File for testing different stuff for the project :D

In [1]:
# imports
import pandas as pd
import numpy as np
import pprint

# Testing on reading one of the datasets and creating a Pandas DataFrame - object
test_df = pd.read_csv("HybridDataset_for_RSSbasedLocalization-main/Scenario 0/RPI/RSS_BLT_Dataset/Distanza10.csv", sep=';')
test_df.head()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
test_df.info()

: 

Everything seemed to be working great as we got the first five entries of the dataset with a .head()– method and some information about the dataset with the .info()– method.

In [ ]:
def read_data(anchors, directory_path):
    """
    Reads the csv-files for one specified measurement case and adds them into a dict of dataframes.
    
    Parameters:
    anchors (list): List of anchors, for example ["Anchor 1", "Anchor 2", "Anchor 3"...]
    directory path (string): The relative directory path of the folder which contains the wanted anchor-csv.files.

    returns: 
    scenario_dataframes (dict): a dict which keys are the anchor names and values are the respective dataframes.
    """
    scenario_dataframes = {}

    for anchor in anchors:
        current_df = pd.read_csv(directory_path + anchor + ".csv", sep=';')
        scenario_dataframes[anchor.replace(".csv","")] = current_df
    
    return scenario_dataframes


: 

In [ ]:
def calculate_smallest_dataset(dataframes):
    """
    Calculates the minimum over all dataframes' length.

    Parameters:
    dataframes (dict): a dict which keys are the anchor names and values are the respective dataframes.

    Returns:
    minimum_length (int): an integer which value represents the minimum over all dataframes' length.
    """

    sizes = []

    for anchor in dataframes:
        df = dataframes[anchor]
        sizes.append(len(df))

    return min(sizes)
        

: 

In [ ]:
def calculate_mean_values(scenario_dataframe, length): 
    """
    Calculates the mean value over a given amount of bursts.

    Parameters:
    scenario_dataframe (DataFrame): A DataFrame-object containing the dataset of one anchor.
    length (int): an integer which value represents the number of data to read in the dataset.

    Returns:
    mean_values (list): Returns a list of mean values over the given amount of bursts.
    """
    
    mean_values = []

    power_values = scenario_dataframe["Rx Power [dBm]"].to_list()
    bursts = length // 10
    burst_size = length // bursts

    # print("length: ", length, ", bursts: ", bursts, ", burst_size: ", burst_size)

    start = 0
    for i in range(bursts):
        # end = start + burst_size if i < bursts - 1 else length - 1
        end = start + burst_size
        # print("start: ", start, ", end: ", end)
        # print("power_values: ", power_values[start:end])
        mean_values.append(round(np.mean(power_values[start:end]), 4))
        start = end

    if (start < length):
        end = length
        # print("start: ", start, ", end: ", end)
        mean_values.append(round(np.mean(power_values[start:end]), 4))
        start = end
    
    return mean_values

: 

In [ ]:
def calculate_estimates(p_initial, n_value, mean_values):
    """
    Calculates the distance estimates for each anchor.

    Parameteres:
    p_initial (int): The initial RSS value for the distance estimation equation.
    n_value (int): The n_value for the distance estimation formula.
    mean_values (list): A list which values are mean values over the given amount of bursts.

    Returns:
    estimated_distances (list): Returns a list which values are the distance estimations.
    """
    
    estimated_distances = []

    index = 0
    for mean in mean_values:
        num = mean - p_initial
        den = 10 * n_value
        estimated_distances.append(10**(num/den))
        index = index + 1

    return estimated_distances

: 

Let's estimate the distances of the anchors in Scenario A RPI BLT_dataset

In [ ]:
directory_path = "HybridDataset_for_RSSbasedLocalization-main/Scenario A/RPI/RSS_BLT_Dataset/"
anchors = ["Anchor 1", "Anchor 2", "Anchor 3", "Anchor 4", "Anchor 5", "Anchor 6"]

mean_values = {}
estimated_distances = {}
configurations = {}

p_initial = -40
n_value = 4

dataframes = read_data(anchors, directory_path)
length = calculate_smallest_dataset(dataframes)

for anchor in dataframes:
    
    mean_values[anchor] = calculate_mean_values(dataframes[anchor], length)
    estimated_distances[anchor] = calculate_estimates(p_initial, n_value, mean_values[anchor])
    
    # print("--- ", anchor, " ---")
    # print("")
    # print("Mean values: ", mean_values[anchor])
    # print("")
    # print("Estimated distances:", estimated_distances[anchor])
    # print("")
    # print("-----")
    # print("")

for index in range(len(mean_values["Anchor 1"])):
    i = index + 1
    configurations[i] = {}
    configurations[i]["Mean Power"] = []
    configurations[i]["Estimated Distance"] = []
    for anchor in dataframes:
        configurations[i]["Mean Power"].append(mean_values[anchor][index])
        configurations[i]["Estimated Distance"].append(estimated_distances[anchor][index])
        configurations[i]["Target Position"] = dataframes[anchor]["Distance Target - Anchor [m]"][0]

: 

In [ ]:
pprint.pprint(configurations)

: 

: 